In [1]:
#change this to gether the TCX files since they have the timezone info
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'dataImport/')
from utils import workingDataPath, getWorkingHRDfParquet, exportsDataPath, writeWorkingHRDfParquet
import pandas as pd

workingDataFile = workingDataPath + "/polar/hr/"
pathOfExport = exportsDataPath + "polar/individualSessions/"
dir_list = os.listdir(pathOfExport)

In [2]:
def dfFromFile(fileName):
    rawDf = pd.read_csv(pathOfExport + fileName)
    columnNames = ["sampleDT", "value"]

    startDT = pd.to_datetime(" ".join([rawDf["Date"].loc[0], rawDf["Start time"].loc[0],"-0700"]))
    times = startDT + pd.to_timedelta(rawDf["Sport"].loc[2:]).values
    hrs = rawDf["Date"].loc[2:].apply(lambda x:0 if pd.isna(x) else int(x)).values

    samplesList = [[times[i], hrs[i]] for i in range(len(times))]
    samplesList = [x for x in samplesList if x[1] > 0]
    samplesList = sorted(samplesList, key=lambda x:x[0])


    dataDf = pd.DataFrame(data=samplesList, columns=columnNames)
    return dataDf.set_index("sampleDT")

In [3]:
dfToAdd = pd.DataFrame(columns = ["sampleDT", "value"])
dfToAdd = dfToAdd.set_index('sampleDT')

for fn in dir_list:
    dfToAdd = pd.concat([dfToAdd, dfFromFile(fn)])
dfToAdd = dfToAdd[~dfToAdd.index.duplicated(keep='first')]
dfToAdd

/home/chowder/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


,value
sampleDT,
2024-11-09 23:40:11-07:00,58
2024-11-09 23:40:12-07:00,58
2024-11-09 23:40:13-07:00,59
2024-11-09 23:40:14-07:00,59
2024-11-09 23:40:15-07:00,59
...,...
2024-07-10 10:34:33-07:00,84
2024-07-10 10:34:34-07:00,85
2024-07-10 10:34:35-07:00,85


In [4]:
# read working data
fullDF = pd.concat([dfToAdd, getWorkingHRDfParquet('polar')])
fullDF["value"] = fullDF["value"].astype("int")
fullDF = fullDF[~fullDF.index.duplicated(keep='first')]
fullDF

,value
sampleDT,
2024-11-09 23:40:11-07:00,58
2024-11-09 23:40:12-07:00,58
2024-11-09 23:40:13-07:00,59
2024-11-09 23:40:14-07:00,59
2024-11-09 23:40:15-07:00,59
...,...
2024-08-07 22:54:04-07:00,82
2024-08-07 22:54:05-07:00,82
2024-08-07 22:54:06-07:00,83


In [5]:
writeWorkingHRDfParquet('polar', fullDF)

the file size of all the data is about 8 MB
the total number of rows in the file is 1903183
splitting into 2 number of about 5MB files with 951591 rows per file
saving rows 0 to 951590
value    58
Name: 2024-11-09 23:40:11-07:00, dtype: int64
to a file named 2024-11-09T234011-0700_2024-09-13T231021-0700.parquet.gzip
2024-11-09 23:40:11-07:00
saving rows 951591 to 1903181
value    70
Name: 2024-09-13 23:10:22-07:00, dtype: int64
to a file named 2024-09-13T231022-0700_2024-08-07T225407-0700.parquet.gzip
2024-09-13 23:10:22-07:00
saving rows 1903182 to 1903182
value    83
Name: 2024-08-07 22:54:08-07:00, dtype: int64
to a file named 2024-08-07T225408-0700_2024-08-07T225408-0700.parquet.gzip
2024-08-07 22:54:08-07:00
